# Data Engineering in Python with databolt - Fuzzy Joins (d6tlib/d6tjoin.utils)

## Introduction

Joining datasets is a common data engineering operation. However, often there are problems merging datasets from different sources because of mismatched identifiers, date conventions etc. 

** `d6tjoin.top1` module allows you to quickly join datasets even if they don't perfectly match. **
Easily join different datasets without writing custom code. Does fuzzy top1 similarity joins for strings, dates and numbers, for example you can quickly join similar but not identical stock tickers, addresses, names without manual processing. It will find the top 1 matched entry from the right dataframe to join onto the left dataframe.

Here are some examples which show you how to:
1. join on mismatched identifiers
2. join on calendar vs business dates
3. join on both mismatched dates and identifiers

In [28]:
import pandas as pd
import numpy as np
import itertools
from faker import Faker
import importlib

import d6tjoin.top1
importlib.reload(d6tjoin.top1)
import d6tjoin.utils

# *******************************************************
# generate sample time series data with id and value
# *******************************************************
nobs = 10
f1 = Faker()
f1.seed(0)
uuid1 = [str(f1.uuid4()).split('-')[0] for _ in range(nobs)]
dates1 = pd.date_range('1/1/2010','1/1/2011')

df1 = pd.DataFrame(list(itertools.product(dates1,uuid1)),columns=['date','id'])
df1['v']=np.round(np.random.sample(df1.shape[0]),3)
df1.head()

,date,id,v
0,2010-01-01,e3e70682,0.271
1,2010-01-01,f728b4fa,0.576
2,2010-01-01,eb1167b3,0.038
3,2010-01-01,f7c1bd87,0.784
4,2010-01-01,e443df78,0.387


# Example 1: join datasets on misalgined ids

When joining data from different sources, eg different vendors, often your ids don't match perfect and then you need to manually analyze the situation. With databolt this becomes much easier.

Let's create another dataset where the `id` is slightly different.

In [29]:
# create mismatch
df2 = df1.copy()
df2['id'] = df1['id'].str[1:-1]
df2.head()

,date,id,v
0,2010-01-01,3e7068,0.271
1,2010-01-01,728b4f,0.576
2,2010-01-01,b1167b,0.038
3,2010-01-01,7c1bd8,0.784
4,2010-01-01,443df7,0.387


`d6tjoin.utils.PreJoin().stats_prejoin` shows you there is none of `id` match so a normal join won't work well.

In [30]:
d6tjoin.utils.PreJoin([df1,df2],['id','date']).stats_prejoin(print_only=False)

,key left,key right,all matched,inner,left,right,outer,unmatched total,unmatched left,unmatched right
0,id,id,False,0,10,10,20,20,10,10
1,date,date,True,366,366,366,366,0,0,0
2,__all__,__all__,False,0,3660,3660,7320,7320,3660,3660


Using `d6tjoin.top1.MergeTop1()` you can quickly merge this dataset without having to do any manual processing. It will find the closest matching id using the Levenstein string similarity metric. We want to look at the closest id by date so we will pass in date as an exact match key.

In [31]:
result = d6tjoin.top1.MergeTop1(df1.head(),df2,fuzzy_left_on=['id'],fuzzy_right_on=['id'],exact_left_on=['date'],exact_right_on=['date']).merge()

Lets check what matches it found. Looking at the top1 match table, it shows the closest string with only 2 character difference in id, meaning it found the correct substring. 

In [32]:
result['top1']['id']

,date,__top1left__,__top1right__,__top1diff__,__matchtype__
0,2010-01-01,e3e70682,3e7068,2,top1 left
44,2010-01-01,e443df78,443df7,2,top1 left
32,2010-01-01,eb1167b3,b1167b,2,top1 left
11,2010-01-01,f728b4fa,728b4f,2,top1 left
23,2010-01-01,f7c1bd87,7c1bd8,2,top1 left


Since the match results look good, you can use the merged dataset.

In [33]:
result['merged'].head()

,date,id,v,id_right,v_right
0,2010-01-01,e3e70682,0.271,3e7068,0.271
1,2010-01-01,f728b4fa,0.576,728b4f,0.576
2,2010-01-01,eb1167b3,0.038,b1167b,0.038
3,2010-01-01,f7c1bd87,0.784,7c1bd8,0.784
4,2010-01-01,e443df78,0.387,443df7,0.387


In [34]:
assert not result['duplicates']

# Example 2: join 2 datasets with misalgined dates

As another example, instead of the ids not matching, lets look at an example where the dates don't match. We will look at calendar vs business month end dates.

In [35]:
dates2 = pd.bdate_range('1/1/2010','1/1/2011') # business instead of calendar dates
df2 = pd.DataFrame(list(itertools.product(dates2,uuid1)),columns=['date','id'])
df2['v']=np.round(np.random.sample(df2.shape[0]),3)

`d6tjoin.utils.PreJoin()` shows some but not all of the dates match. All the ids match.

In [36]:
d6tjoin.utils.PreJoin([df1,df2],['id','date']).stats_prejoin(print_only=False)

,key left,key right,all matched,inner,left,right,outer,unmatched total,unmatched left,unmatched right
0,id,id,True,10,10,10,10,0,0,0
1,date,date,False,261,366,261,366,105,105,0
2,__all__,__all__,False,2610,3660,2610,3660,1050,1050,0


So we want to do a fuzzy match on dates but have the id match perfectly.

In [37]:
result = d6tjoin.top1.MergeTop1(df1,df2,fuzzy_left_on=['date'],fuzzy_right_on=['date'],exact_left_on=['id'],exact_right_on=['id']).merge()

Again lets check if the fuzzy matches are correct. If either matches or is off by a day most, looks good!

In [38]:
result['top1']['date'].head(3)

,id,__top1left__,__top1right__,__top1diff__,__matchtype__
0,1846d424,2010-01-01,2010-01-01,0 days,exact
1,eb1167b3,2010-01-01,2010-01-01,0 days,exact
2,e443df78,2010-01-01,2010-01-01,0 days,exact


In [39]:
result['top1']['date'].tail(3)

,id,__top1left__,__top1right__,__top1diff__,__matchtype__
3657,1846d424,2011-01-01,2010-12-31,1 days,top1 left
3658,f7c1bd87,2011-01-01,2010-12-31,1 days,top1 left
3659,fcbd04c3,2011-01-01,2010-12-31,1 days,top1 left


In [40]:
result['top1']['date']['__top1diff__'].max()

Timedelta('1 days 00:00:00')

Again with very little effort we were able to join this dataset together.

In [41]:
result['merged'].head()

,date,id,v,date_right,v_right
0,2010-01-01,e3e70682,0.271,2010-01-01,0.864
1,2010-01-02,e3e70682,0.010,2010-01-01,0.864
2,2010-01-01,f728b4fa,0.576,2010-01-01,0.196
3,2010-01-02,f728b4fa,0.524,2010-01-01,0.196
4,2010-01-01,eb1167b3,0.038,2010-01-01,0.950


# Example 3: join 2 datasets with misalgined dates AND ids

In the final example, we combine the above cases. None of the ids match and some of the dates are mismatched. As before with little manual effort we are able to correctly merge the dataset.

In [42]:
dates2 = pd.bdate_range('1/1/2010','1/1/2011') # business instead of calendar dates
df2 = pd.DataFrame(list(itertools.product(dates2,uuid1)),columns=['date','id'])
df2['v']=np.round(np.random.sample(df2.shape[0]),3)
df2['id'] = df2['id'].str[1:-1]

In [43]:
d6tjoin.utils.PreJoin([df1,df2],['id','date']).stats_prejoin(print_only=False)

,key left,key right,all matched,inner,left,right,outer,unmatched total,unmatched left,unmatched right
0,id,id,False,0,10,10,20,20,10,10
1,date,date,False,261,366,261,366,105,105,0
2,__all__,__all__,False,0,3660,2610,6270,6270,3660,2610


In [44]:
result = d6tjoin.top1.MergeTop1(df1,df2,['date','id'],['date','id']).merge()

In [45]:
result['merged'].head()

,date,id,v,date_right,id_right,v_right
0,2010-01-01,e3e70682,0.271,2010-01-01,3e7068,0.322
1,2010-01-02,e3e70682,0.010,2010-01-01,3e7068,0.322
2,2010-01-01,f728b4fa,0.576,2010-01-01,728b4f,0.674
3,2010-01-02,f728b4fa,0.524,2010-01-01,728b4f,0.674
4,2010-01-01,eb1167b3,0.038,2010-01-01,b1167b,0.150


In [46]:
result['top1']['date'].tail()

,__top1left__,__top1right__,__top1diff__,__matchtype__
361,2010-12-28,2010-12-28,0 days,exact
362,2010-12-29,2010-12-29,0 days,exact
363,2010-12-30,2010-12-30,0 days,exact
364,2010-12-31,2010-12-31,0 days,exact
365,2011-01-01,2010-12-31,1 days,top1 left


In [47]:
result['top1']['id'].head()

,__top1right__date,__top1left__,__top1right__,__top1diff__,__matchtype__
17226,2010-01-01,1846d424,846d42,2,top1 left
14355,2010-01-01,23a7711a,3a7711,2,top1 left
10179,2010-01-01,259f4329,59f432,2,top1 left
20358,2010-01-01,b4862b21,4862b2,2,top1 left
20880,2010-01-01,e3e70682,3e7068,2,top1 left


# Advanced Usage Options

## Passing a difference limit
By default every record in the left dataframe will be matched with a record in the right dataframe. Sometimes the difference is too large though to be considered a match. You can control this by passing the `top_limit` parameter.

In [48]:
dates2 = pd.bdate_range('1/1/2010','1/1/2011') # business instead of calendar dates
df2 = pd.DataFrame(list(itertools.product(dates2,uuid1[:-2])),columns=['date','id'])
df2['v']=np.random.sample(df2.shape[0])
df2['id'] = df2['id'].str[1:-1]

In [49]:
result = d6tjoin.top1.MergeTop1(df1,df2,['date','id'],['date','id']).merge()
result['top1']['id'].head()

,__top1right__date,__top1left__,__top1right__,__top1diff__,__matchtype__
14094,2010-01-01,1846d424,846d42,2,top1 left
11745,2010-01-01,23a7711a,3a7711,2,top1 left
7830,2010-01-01,259f4329,846d42,6,top1 left
15138,2010-01-01,b4862b21,b1167b,5,top1 left
16182,2010-01-01,b4862b21,846d42,5,top1 left


We have some correct matches but also some bad matches with `__top1diff__`>2. We will restrict `top_limit` to be at most 2.

In [50]:
result = d6tjoin.top1.MergeTop1(df1,df2,['date','id'],['date','id'], top_limit=[None,2]).merge()

In [51]:
result['top1']['id'].head()

,__top1right__date,__top1left__,__top1right__,__top1diff__,__matchtype__
14094,2010-01-01,1846d424,846d42,2,top1 left
11745,2010-01-01,23a7711a,3a7711,2,top1 left
16704,2010-01-01,e3e70682,3e7068,2,top1 left
5220,2010-01-01,e443df78,443df7,2,top1 left
19314,2010-01-01,eb1167b3,b1167b,2,top1 left


## Passing a custom difference function
By default string matches are done using Levenstein edit distance. You can pass a custom function using `fun_diff`. For example lets pass Hamming distance.

In [52]:
import jellyfish
result = d6tjoin.top1.MergeTop1(df1,df2,['date','id'],['date','id'], fun_diff=[None,jellyfish.hamming_distance]).merge()

In [53]:
result['top1']['id'].head()

,__top1right__date,__top1left__,__top1right__,__top1diff__,__matchtype__
13050,2010-01-01,1846d424,b1167b,7,top1 left
13311,2010-01-01,1846d424,7c1bd8,7,top1 left
11745,2010-01-01,23a7711a,3a7711,6,top1 left
6525,2010-01-01,259f4329,728b4f,7,top1 left
7830,2010-01-01,259f4329,846d42,7,top1 left
